In [2]:
!pip install transformers datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 23.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.6/761.6 kB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.3/436.3 kB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 70.3 MB/s eta 0:00:00
 

In [3]:
import torch
import torch.nn as nn
from transformers import (
    BertConfig,
    BertModel,
    BertForMaskedLM,
    BertTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from datasets import load_dataset
import copy

/home/I6356345/anaconda3/envs/p37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
class BertConfigRelativePositional(BertConfig):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.position_embedding_type = "relative_key_query"

In [4]:
def rotate_every_two(x):
    x1 = x[:, :, :, ::2]
    x2 = x[:, :, :, 1::2]
    x = torch.stack((-x2, x1), dim=-1)
    return x.flatten(-2)

def apply_rotary_positional_embeddings(x, sinusoidal_pos):
    return (x * sinusoidal_pos.cos()) + (rotate_every_two(x) * sinusoidal_pos.sin())

In [5]:
from transformers.models.bert.modeling_bert import BertAttention, BertLayer, BertEncoder

class BertAttentionWithRotary(BertAttention):
    def forward(self, hidden_states, attention_mask=None, head_mask=None, output_attentions=False):
        # Original attention code here
        # Apply rotary positional embeddings before attention calculation
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        sinusoidal_pos = self.get_sinusoidal_pos(hidden_states)
        query_layer = apply_rotary_positional_embeddings(query_layer, sinusoidal_pos)
        key_layer = apply_rotary_positional_embeddings(key_layer, sinusoidal_pos)
        # Continue with the original attention computation
        # ...

class BertLayerCustom(BertLayer):
    def __init__(self, config):
        super().__init__(config)
        if config.position_embedding_type == "rotary":
            self.attention = BertAttentionWithRotary(config)

In [6]:
from transformers.models.bert.modeling_bert import BertModel

class BertModelCustom(BertModel):
    def __init__(self, config):
        super().__init__(config)
        if config.position_embedding_type in ["relative_key_query", "rotary"]:
            self.encoder = BertEncoderCustom(config)

class BertEncoderCustom(BertEncoder):
    def __init__(self, config):
        super().__init__(config)
        self.layer = nn.ModuleList([BertLayerCustom(config) for _ in range(config.num_hidden_layers)])


In [7]:
# Load a small dataset for demonstration purposes
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], return_special_tokens_mask=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

In [10]:
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=256,
    num_hidden_layers=4,
    num_attention_heads=4,
    max_position_embeddings=512,
)

# Choose the type of positional embeddings
config.position_embedding_type = 'absolute'  # Options: 'absolute', 'relative_key_query', 'rotary'

model = BertForMaskedLM(config)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [11]:
# Freeze embeddings and first two layers
for param in model.bert.embeddings.parameters():
    param.requires_grad = False

for idx, layer in enumerate(model.bert.encoder.layer):
    if idx < 2:
        for param in layer.parameters():
            param.requires_grad = False

In [12]:
# Modify the forward method to output hidden states
config.output_hidden_states = True

# After training, get outputs and probe layers
def probe_model(input_ids):
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs.hidden_states  # Tuple of hidden states from all layers
    return hidden_states

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/I6356345/anaconda3/lib/python3.12/site-packages/torch/nn/parallel/data_parallel.py:34: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/I6356345/anaconda3/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py", line 84, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/I6356345/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/I6356345/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/I6356345/anaconda3/lib/python3.12/site-packages/transformers/models/bert/modeling_bert.py", line 1507, in forward
    prediction_scores = self.cls(sequence_output)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/I6356345/anaconda3/lib/python3.12/site-p

In [14]:
# Get a batch of data
sample = next(iter(tokenized_dataset))
input_ids = torch.tensor([sample['input_ids']])

# Probe the model
hidden_states = probe_model(input_ids)

# Analyze the hidden states from different layers
for idx, layer_hidden_state in enumerate(hidden_states):
    print(f"Layer {idx}: {layer_hidden_state.shape}")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)